# Feature engineering

> In this module, we develop tools to extract features from compounds

In [1]:
#| default_exp feature

In [12]:
#| hide
import sys
sys.path.append("/notebooks/tools")
from tools.dataset import *

In [9]:
#| hide
from nbdev.showdoc import *
%matplotlib inline

In [3]:
#| export
import seaborn as sns
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.ML.Descriptors import MoleculeDescriptors
import pandas as pd
from rdkit.Chem import Draw
from rdkit.Chem import Descriptors
from sklearn.preprocessing import StandardScaler

In [4]:
#| export
from fastbook import *
from fairscale.nn.data_parallel import FullyShardedDataParallel as FSDP
from fairscale.nn.wrap import enable_wrap, wrap
import esm
from tqdm.notebook import tqdm; tqdm.pandas()
import gc

In [5]:
#| export
def smi2prop(df, # df needs to have SMILES an ID columns
             smi_colname = "SMILES", # column name of smiles
             id_colname = "ID", # column name of ID
             remove_duplicate=True, # remove features that are same across compounds
             normalize = False, # normalize features using StandardScaler()
            ):
    "Extract 208 features from smiles via rdkit.Chem.Descriptors, and remove duplicate features"
    
    mols = [Chem.MolFromSmiles(smi) for smi in df[smi_colname]]
    desc_names = [desc_name[0] for desc_name in Descriptors.descList]
    desc_calc = MoleculeDescriptors.MolecularDescriptorCalculator(desc_names)
    desc_values = [desc_calc.CalcDescriptors(mol) for mol in mols]
    compound = pd.DataFrame(np.stack(desc_values).T, index=desc_names,columns=df[id_colname])
    if remove_duplicate:
        compound = compound.loc[~compound.duplicated()] # remove duplicates
        compound = compound.loc[compound.std(axis=1) != 0] # remove compound that has same value across features
    compound = compound.T.reset_index()
    if normalize:
        scaler = StandardScaler()
        transformed = scaler.fit_transform(compound.iloc[:,1:])
        compound.iloc[:,1:] = transformed
    return compound

In [10]:
show_doc(smi2prop)

---

[source](https://github.com/sky1ove/tools/blob/main/tools/feature.py#L18){target="_blank" style="float:right; font-size:smaller"}

### smi2prop

>      smi2prop (df, smi_colname='SMILES', id_colname='ID',
>                remove_duplicate=True, normalize=False)

Extract 208 features from smiles via rdkit.Chem.Descriptors, and remove duplicate features

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df |  |  | df needs to have SMILES an ID columns |
| smi_colname | str | SMILES | column name of smiles |
| id_colname | str | ID | column name of ID |
| remove_duplicate | bool | True | remove features that are same across compounds |
| normalize | bool | False | normalize features using StandardScaler() |

In [13]:
df = get_g12d()

(722, 10) ['ID', 'SMILES', 'group', 'with_3F', 'racemic_trans', 'mixture_isomer', 'trans', 'Kd', 'IC50', 'erk_IC50']


In [14]:
df.shape

(722, 10)

In [15]:
smi2prop(df,normalize=True)

,ID,MaxEStateIndex,MinEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,MaxPartialCharge,MinPartialCharge,MaxAbsPartialCharge,MinAbsPartialCharge,FpDensityMorgan1,FpDensityMorgan2,FpDensityMorgan3,BCUT2D_MWHI,BCUT2D_MWLOW,BCUT2D_CHGHI,BCUT2D_CHGLO,BCUT2D_LOGPHI,BCUT2D_LOGPLOW,BCUT2D_MRHI,BCUT2D_MRLOW,BalabanJ,BertzCT,Chi0,Chi0n,Chi0v,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3n,Chi3v,Chi4n,Chi4v,HallKierAlpha,Ipc,Kappa1,Kappa2,Kappa3,LabuteASA,PEOE_VSA1,PEOE_VSA10,PEOE_VSA11,PEOE_VSA12,PEOE_VSA13,PEOE_VSA14,PEOE_VSA2,PEOE_VSA3,PEOE_VSA4,PEOE_VSA5,PEOE_VSA6,PEOE_VSA7,PEOE_VSA8,PEOE_VSA9,SMR_VSA1,SMR_VSA10,SMR_VSA2,SMR_VSA3,SMR_VSA4,SMR_VSA5,SMR_VSA6,SMR_VSA7,SMR_VSA9,SlogP_VSA1,SlogP_VSA10,SlogP_VSA11,SlogP_VSA12,SlogP_VSA2,SlogP_VSA3,SlogP_VSA4,SlogP_VSA5,SlogP_VSA6,SlogP_VSA7,SlogP_VSA8,TPSA,EState_VSA1,EState_VSA10,EState_VSA11,EState_VSA2,EState_VSA3,EState_VSA4,EState_VSA5,EState_VSA6,EState_VSA7,EState_VSA8,EState_VSA9,VSA_EState1,VSA_EState10,VSA_EState2,VSA_EState3,VSA_EState4,VSA_EState5,VSA_EState6,VSA_EState7,VSA_EState8,VSA_EState9,FractionCSP3,HeavyAtomCount,NHOHCount,NOCount,NumAliphaticCarbocycles,NumAliphaticHeterocycles,NumAliphaticRings,NumAromaticCarbocycles,NumAromaticHeterocycles,NumAromaticRings,NumHAcceptors,NumHDonors,NumHeteroatoms,NumRotatableBonds,NumSaturatedCarbocycles,NumSaturatedHeterocycles,NumSaturatedRings,RingCount,MolLogP,MolMR,fr_Al_COO,fr_Al_OH,fr_Al_OH_noTert,fr_ArN,fr_Ar_N,fr_Ar_NH,fr_Ar_OH,fr_COO2,fr_C_O,fr_C_O_noCOO,fr_HOCCN,fr_Imine,fr_NH0,fr_NH1,fr_NH2,fr_N_O,fr_Ndealkylation1,fr_Ndealkylation2,fr_aldehyde,fr_alkyl_carbamate,fr_alkyl_halide,fr_allylic_oxid,fr_amide,fr_aniline,fr_aryl_methyl,fr_bicyclic,fr_ester,fr_ether,fr_guanido,fr_halogen,fr_imidazole,fr_lactone,fr_methoxy,fr_morpholine,fr_nitrile,fr_para_hydroxylation,fr_piperdine,fr_piperzine,fr_priamide,fr_pyridine,fr_quatN,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_thiazole,fr_unbrch_alkane,fr_urea
0,US_1,-1.932028,0.932171,1.381110,0.495241,-1.183587,-1.219109,-1.182076,-1.046640,-0.203173,-1.070885,0.960805,-0.201299,-0.606818,-0.027786,0.238330,-0.929849,0.169612,-1.291745,0.519884,-1.519547,0.185658,-0.425142,0.608875,0.337035,-0.603426,-1.036679,-0.801448,-0.917005,-0.932184,-0.777188,-0.867430,-0.807099,-0.902348,-0.806015,-0.900582,-0.864011,-0.972370,-0.126939,-0.154995,-1.075533,-0.754664,-0.327962,-0.976340,0.813537,-1.109378,-1.722754,-0.237511,-0.150438,-0.258243,-0.690788,-1.677478,-0.252705,-0.275917,-0.455194,0.586341,-0.695775,0.424056,-0.818465,-0.302768,-0.193272,0.123156,-0.361079,-0.742411,-0.199686,-0.279138,0.655344,-0.36044,-1.709988,0.908283,-0.727266,0.045041,-0.772241,-1.454431,-0.791409,0.740918,-0.673271,0.606634,0.257682,-1.120582,-0.123426,-1.959448,-1.622800,-1.137956,0.581413,0.436907,-0.082673,0.600444,-0.316618,0.410895,-1.477411,-0.713491,-0.020944,0.966588,0.316115,0.706608,1.074237,0.035838,-0.307140,0.688654,-0.495195,-0.975006,0.504157,0.034921,-0.393,-0.446513,-0.613431,0.241193,0.123187,0.298974,0.164340,0.642889,-1.190554,-0.645101,-0.356435,-0.308046,-0.465984,-0.535647,-0.875193,-0.643206,-0.037242,-0.289541,-0.278433,-0.145659,-0.016211,-0.164399,1.260085,-0.052705,-0.321026,-0.31841,-0.087167,-0.037242,-0.098784,-0.261324,-0.215353,-0.051678,1.432247,1.722508,-0.091542,-0.14062,-0.508104,-0.070507,-0.309352,-0.493979,-0.51257,-0.342359,-0.052705,-0.316228,-0.052705,-1.583939,-0.204565,-0.037242,-0.258067,-0.145659,-0.185429,-0.211808,-0.183176,0.327661,-0.052705,0.357590,-0.037242,-0.083507,-0.037242,-0.052705,-0.204565,-0.091542,-0.16413,-0.037242
1,US_2,0.370290,0.282627,-0.324387,0.318811,-0.840874,-0.837744,-0.838700,-0.740120,-0.200922,-1.070885,0.960805,-0.198749,0.203344,0.332533,0.322539,-0.646890,0.160857,-1.217596,0.513626,-0.927388,0.182332,-0.416530,0.608299,0.705937,-0.323952,-0.697765,-0.663958,-0.781950,-0.685520,-0.697979,-0.791224,-0.730912,-0.830814,-0.734399,-0.831915,-0.806103,-0.915471,-0.260409,-0.153546,-0.753847,-0.57500

In [ ]:
smi2prop(df,normalize=False)

,ID,MaxEStateIndex,MinEStateIndex,MinAbsEStateIndex,qed,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,MaxPartialCharge,...,fr_halogen,fr_imidazole,fr_methoxy,fr_morpholine,fr_nitrile,fr_piperdine,fr_piperzine,fr_pyridine,fr_term_acetylene,fr_unbrch_alkane
0,G12D_1,16.725593,-0.912746,0.007777,0.319044,600.645,569.397,600.246059,226.0,0.318621,...,3.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0
1,G12D_5A,16.408797,-0.546191,0.013725,0.369319,546.050,516.818,545.210614,200.0,0.318610,...,2.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,G12D_5B,16.254417,-0.498575,0.197532,0.426624,507.013,478.789,506.199715,186.0,0.318610,...,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
3,G12D_6,16.215868,-0.551045,0.054533,0.441060,488.567,459.335,488.233602,186.0,0.318610,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,G12D_7,16.296115,-0.548440,0.055691,0.424225,502.594,471.346,502.249252,192.0,0.318610,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,G12C_20,13.794964,-1.016664,0.079151,0.358213,604.130,568.850,603.252479,224.0,0.318223,...,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
58,G12C_21,12.978753,-0.325754,0.071299,0.269545,616.166,577.862,615.272466,230.0,0.318223,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
59,G12C_22,12.777688,-0.705241,0.172589,0.334036,618.157,580.861,617.268129,230.0,0.318223,...,2.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
60,G12C_23,13.004729,-0.260666,0.105776,0.318010,630.193,589.873,629.288116,236.0,0.318223,...,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [7]:
#| export
def smi2morgan(df, # a dataframe contains ID and SMILES columns
               smi_colname = "SMILES", # set smiles columne name
               id_colname = "ID", # set ID column name
              ):
    "Like `smi2prop`, get 2048 morgan feature (0/1) given a dataframe that contains ID&smiles"
    mols = [Chem.MolFromSmiles(smi) for smi in df[smi_colname]]
    morgan_fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048) for mol in mols]
    fp_df = pd.DataFrame(np.array(morgan_fps), index=df[id_colname])
    colnames = [f'morgan_{i}' for i in fp_df.columns]
    fp_df.columns = colnames
    fp_df = fp_df.reset_index()
    return fp_df

In [11]:
show_doc(smi2morgan)

---

[source](https://github.com/sky1ove/tools/blob/main/tools/feature.py#L42){target="_blank" style="float:right; font-size:smaller"}

### smi2morgan

>      smi2morgan (df, smi_colname='SMILES', id_colname='ID')

Like `smi2prop`, get 2048 morgan feature (0/1) given a dataframe that contains ID&smiles

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| df |  |  | a dataframe contains ID and SMILES columns |
| smi_colname | str | SMILES | set smiles columne name |
| id_colname | str | ID | set ID column name |

In [ ]:
smi2morgan(df)

,ID,morgan_0,morgan_1,morgan_2,morgan_3,morgan_4,morgan_5,morgan_6,morgan_7,morgan_8,...,morgan_2038,morgan_2039,morgan_2040,morgan_2041,morgan_2042,morgan_2043,morgan_2044,morgan_2045,morgan_2046,morgan_2047
0,G12D_1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,G12D_5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,G12D_5B,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,G12D_6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,G12D_7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,G12C_20,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,0,0
58,G12C_21,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
59,G12C_22,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
60,G12C_23,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()